<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/2_rag_texto_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia 1
    * Usar segmentação de sentenças
    * BM25 como buscador

# Instalando pacotes

In [ ]:
!pip install -q llama-index llama-index-llms-groq llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00


# Importando pacotes

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
import json
import time
import random
from tqdm import tqdm
from rich.pretty import pprint

from llama_index.core import Document
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.callbacks import (CallbackManager, LlamaDebugHandler, CBEventType)
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.response.notebook_utils import display_source_node, display_response
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent

from llama_index.llms.groq import Groq

from IPython.display import Markdown, display
import pickle
import pandas as pd

In [ ]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 5812, done.
remote: Counting objects: 100% (817/817), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 5812 (delta 555), reused 756 (delta 505), pack-reused 4995
Receiving objects: 100% (5812/5812), 1.12 GiB | 28.90 MiB/s, done.
Resolving deltas: 100% (1944/1944), done.
Updating files: 100% (5776/5776), done.


In [ ]:
DATA_DIR = '/content/ia024-projeto-rag'

# Funções

In [ ]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [ ]:
%cd /content/ia024-projeto-rag

!git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

!git pull origin main

/content/ia024-projeto-rag
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (5/5), 281.90 KiB | 1.08 MiB/s, done.
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
   c99118d..1660f7a  main       -> origin/main
Updating c99118d..1660f7a
Fast-forward
 "presentations/Constru\303\247\303\243o do Dataset - Projeto Final.pdf" | Bin 0 -> 146304 bytes
 presentations/Plano do Projeto Final.pdf                                | Bin 0 -> 189482 bytes
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 "presentations/Constru\303\247\303\243o do Dataset - Projeto Final.pdf"
 create mode 100644 presentations/Plano do Projeto Final.pdf


## Criando documentos e nós

In [ ]:
from llama_index.core.schema import MetadataMode

docs = []

with open(f"{DATA_DIR}/2-dataset/estatuto.json", 'r') as f:
    estatuto = json.load(f)

metadata={
        "doc_name": estatuto['titulo'],
        "category": "Reitoria"
    }

doc = Document(id_=0
               , text=estatuto['texto_compacto']
               , metadata=metadata
               )

#print(
#    "The LLM sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.LLM),
#)
#print(
#    "The Embedding model sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.EMBED),
#)

docs.append(doc)

In [ ]:
pprint(docs[0])

Document(
│   id_='0',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I. Reitoria, no endereço citado no §1º. II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI. Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. § 3º- Para efeito da incidência das disposições qu

In [ ]:
# Esta classe separa o texto em sentenças e, de acordo com o parâmetro "window_size",
# cria janelas com a sentença atual mais 'window_size' sentenças antes e depois
# as janelas são salvas no metadata "window" e as sentenças em "original_sentence"
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3, # com valor 1, as "janelas" terão tamanho 3: uma sentença antes, a sentença atual, uma sentença depois
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)

nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
len(nodes)

264

In [ ]:
pprint(nodes[0])

TextNode(
│   id_='ad00c752-7898-4884-9a29-938b1e9a5369',
│   embedding=None,
│   metadata={
│   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. ',
│   │   'original_sentence': 'ESTATUTO. ',
│   │   'doc_name': 'Estatuto',
│   │   'category': 'Reitoria'
│   },
│   excluded_embed_metadata_keys=['window', 'original_sentence'],
│   excluded_llm_metadata_keys=['window', 'original_sentence'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='0',
│   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   hash='c958262a924fc6e53db65cce9967b79c93c611568eb1a613ff6791791d3dfe21'
│   │   ),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   node_id='e51a19c2-2380-4781-9e71-41632ef2ff7a',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={
│   │   │   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº.  11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. ',
│   │   │   │   'original_sentence': 'TÍTULO I - DA INSTITUIÇÃO. '
│   │   │   },
│   │   │   hash='8f22f8fe4aba5ca0baa7c6c5cbd7ec7b57a0cf66c22597581eaff1c5e2d7c1d0'
│   │   )
│   },
│   text='ESTATUTO. ',
│   start_char_idx=0,
│   end_char_idx=10,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

# Criando Questões/Respostas

* https://docs.llamaindex.ai/en/stable/examples/evaluation/QuestionGeneration/
* https://docs.llamaindex.ai/en/stable/api_reference/evaluation/dataset_generation/

* https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/llama_dataset/generator.py

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.groq import Groq
import nest_asyncio
nest_asyncio.apply()


In [ ]:
dataset = RagDatasetGenerator.from_documents(documents=docs, llm=Groq(model="llama3-70b-8192"), show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
questions = dataset.generate_questions_from_nodes()


100%|██████████| 12/12 [01:02<00:00,  5.24s/it]


In [ ]:
df = questions.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Here are three diverse questions based on the ...,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,,None,ai (llama3-70b-8192)
1,**Question 1:**,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,,None,ai (llama3-70b-8192)
2,What is the nature of the Instituto Federal de...,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,,None,ai (llama3-70b-8192)
3,Here are three questions based on the provided...,"[Campus Catanduva, situado na Avenida Imperatr...",,None,ai (llama3-70b-8192)
4,**Question 1:** What is the total number of ca...,"[Campus Catanduva, situado na Avenida Imperatr...",,None,ai (llama3-70b-8192)
5,**Answer:** (Students should count the number ...,"[Campus Catanduva, situado na Avenida Imperatr...",,None,ai (llama3-70b-8192)
6,Here are three diverse questions based on the ...,[Portaria; IV. Instrução Normativa; V. Comunic...,,None,ai (llama3-70b-8192)
7,**Question 1**,[Portaria; IV. Instrução Normativa; V. Comunic...,,None,ai (llama3-70b-8192)
8,What are the principles that guide the actions...,[Portaria; IV. Instrução Normativa; V. Comunic...,,None,ai (llama3-70b-8192)
9,Here are three questions based on the provided...,"[realizar pesquisas, estimulando o desenvolvim...",,None,ai (llama3-70b-8192)


In [ ]:
dataset = dataset.generate_dataset_from_nodes()


 92%|█████████▏| 11/12 [00:47<00:05,  5.87s/it]WARNING:llama_index.llms.openai.utils:Retrying llama_index.llms.openai.base.OpenAI._achat in 0.8691086677165463 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hwb708w9ehytjfsd4m90jnkx` on tokens per minute (TPM): Limit 6000, Used 7331, Requested 980. Please try again in 23.116999999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.

100%|██████████| 12/12 [01:24<00:00,  7.00s/it]

100%|██████████| 3/3 [00:11<00:00,  3.68s/it]

100%|██████████| 3/3 [00:30<00:00, 10.05s/it]

100%|██████████| 3/3 [00:36<00:00, 12.12s/it]

100%|██████████| 3/3 [00:52<00:00, 17.34s/it]

100%|██████████| 3/3 [00:13<00:00,  4.51s/it]

100%|██████████| 3/3 [00:30<00:00, 10.00s/it]

100%|██████████| 3/3 [00:33<00:00, 11.03s/it]

100%|██████████| 3/3 [00:32<00:00, 10.92s/it]

100%|██████████|

In [ ]:
dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Here are three diverse questions based on the ...,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,I'm ready to answer your questions based on th...,ai (llama3-70b-8192),ai (llama3-70b-8192)
1,**Question 1: Multiple Choice**,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,"I'm happy to help! However, I don't see a ques...",ai (llama3-70b-8192),ai (llama3-70b-8192)
2,What is the nature of the Instituto Federal de...,[ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO...,"According to Article 1 of the Estatuto, the na...",ai (llama3-70b-8192),ai (llama3-70b-8192)
3,Here are three diverse questions based on the ...,"[Campus Catanduva, situado na Avenida Imperatr...",I'm ready to answer your three diverse questio...,ai (llama3-70b-8192),ai (llama3-70b-8192)
4,**Question 1:** What is the total number of ca...,"[Campus Catanduva, situado na Avenida Imperatr...","According to the Estatuto, there are 29 campus...",ai (llama3-70b-8192),ai (llama3-70b-8192)
5,**Answer:** (To be answered by the student),"[Campus Catanduva, situado na Avenida Imperatr...","I'm happy to help! However, I don't see a spec...",ai (llama3-70b-8192),ai (llama3-70b-8192)
6,Here are three diverse questions based on the ...,[Portaria; IV. Instrução Normativa; V. Comunic...,I'm ready to answer your questions based on th...,ai (llama3-70b-8192),ai (llama3-70b-8192)
7,**Question 1**,[Portaria; IV. Instrução Normativa; V. Comunic...,"I'm happy to help! However, I don't see a spec...",ai (llama3-70b-8192),ai (llama3-70b-8192)
8,What are the principles that guide the actions...,[Portaria; IV. Instrução Normativa; V. Comunic...,"According to Article 4, the principles that gu...",ai (llama3-70b-8192),ai (llama3-70b-8192)
9,Here are three questions based on the context ...,"[realizar pesquisas, estimulando o desenvolvim...",I'm ready to answer your questions based on th...,ai (llama3-70b-8192),ai (llama3-70b-8192)


In [ ]:
df = dataset.to_pandas()
df.to_csv(f"{DATA_DIR}/3-questions_answers/qa-llamaindex-estatuto.csv")

In [ ]:
save_on_git("Adicionando questões/respostas geradas para o Estatuto pelo LlamaIndex")

/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
[main bf3a600] Adicionando questões/respostas geradas para o Estatuto pelo LlamaIndex
 1 file changed, 77 insertions(+)
 create mode 100644 3-questions_answers/qa-llamaindex-estatuto.csv
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 14.31 KiB | 3.58 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/rsabilio/ia024-projeto-rag
   1660f7a..bf3a600  main -> main


# BM25

## Criando BM25

In [ ]:
# Criando o BM25
# com 'tokenizer=None', ele vai fazer a remoção de stop words e o stemming
retriever = BM25Retriever(nodes=nodes, tokenizer=None, similarity_top_k=10)

In [ ]:
# Testando a busca com BM25
retriever_nodes = retriever.retrieve("Qual o nome do IFSP?")
for node in retriever_nodes:
    display_source_node(node)

**Node ID:** 5cc235b3-f865-422d-958d-0d3a1c055878<br>**Similarity:** 0.0<br>**Text:** §8º- Os procedimentos de trabalho do Conselho Superior serão disciplinados no seu regulamento o q...<br>

**Node ID:** 5374a138-a569-4ac4-ac64-e3a8ba044ea0<br>**Similarity:** 0.0<br>**Text:** Subseção I: Da Auditoria Interna Art14 - A Unidade de Auditoria Interna é órgão de assessoramento...<br>

**Node ID:** 3f62b5a4-1c62-4b69-a4f6-d403da8975a1<br>**Similarity:** 0.0<br>**Text:** Atos Administrativos do IFSP.<br>

**Node ID:** d700f906-a83d-415a-af53-d934517735b2<br>**Similarity:** 1.0041925000614276<br>**Text:** As competências específicas estão definidas no Regimento-Geral do IFSP.<br>

**Node ID:** 752d9393-d694-44c5-a826-e44b6e73b4a3<br>**Similarity:** 0.0<br>**Text:** apreciar os assuntos de interesse da administração do IFSP a ele submetido.<br>

**Node ID:** 48f69994-2183-40eb-b2eb-e21c5ee37710<br>**Similarity:** 1.1821298093422055<br>**Text:** Art 8º - A organização geral do IFSP compreende: I. ÓRGÃOS SUPERIORES a) Conselho Superior; 1.<br>

**Node ID:** 4b02ecbc-7c51-4cd9-a688-6a63a7fd7b57<br>**Similarity:** 0.787010506601938<br>**Text:** Art 3º - Os atos administrativos do IFSP obedecerão à forma de: I. Resolução; II.<br>

**Node ID:** 3c81c03b-a4b1-49d5-8e40-9cb6007f0a8e<br>**Similarity:** 0.0<br>**Text:** Art 44 - O IFSP expedirá e registrará seus diplomas em conformidade com o §3º do art.<br>

**Node ID:** ffad13a8-3d8e-4b6d-a681-f7fb82ba81e8<br>**Similarity:** 0.0<br>**Text:** propor a criação e alteração de funções e órgãos administrativos da estrutura organizacional do I...<br>

**Node ID:** 5dc6722e-79ac-4fe9-b66b-1102af22bfa1<br>**Similarity:** 0.0<br>**Text:** Art 46 - O IFSP poderá conferir títulos de Mérito Acadêmico, conforme disciplinado no Regimento-G...<br>

## Criando o 'motor de busca'

In [ ]:
# Este motor de busca será utilizado como tool pelo agente

# Após testes e análise do código, percebi que:
# - O BM25 faz a busca utilizando a sentença e não a janela
# - O resultado da busca é substituído pelas respectivas "janelas"
# - O LLM indicado é utilizado para "sintetizar" o conteúdo das janelas
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=Groq(model="llama3-70b-8192"),
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)

In [ ]:
# Testando a busca
response = query_engine.query(
    "Qual o nome do IFSP?"
    #"Quais são os campi do IFSP?"
)
display_response(response)

**`Final Response:`** Não há menção explícita ao nome do IFSP no contexto fornecido.

In [ ]:
for node in response.source_nodes:
    display_source_node(node, source_length=500)

In [ ]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"\n\n**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))


display_prompt_dict(query_engine.get_prompts())

# Criando Agente

In [ ]:
# Definindo as tools disponíveis
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="search",
            description=(
                "Provides context to answer the questions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
# Criando o agente
llm   = Groq(model="llama3-70b-8192")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True # mostra as mensagens na saída
)

In [ ]:
# Testando o agente

agent.reset() # apaga o histórico para executar a próxima consulta

response = agent.chat(
        #"Qual o nome do IFSP?"                                                                  # Conseguiu Responder
        #"Quais são os campi do IFSP?"                                                           # Respondeu parcialmente, pois não listou todos os campi
        #"O IFSP tem só ensino médio? Informe o documento, o Capítulo e o Artigo de referência." # Só conseguiu falar que é o Estatuto e o Art 5, mas errou o Capítulo
        #"O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência." # Respondeu, mas não falou a referência
        #"Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"              # Estourou o número de interações e não conseguiu responder
        "O que diz o Art 6º do estatuto do IFSP?"                                                # Conseguiu responder com 'Art 6º', mas não com "artigo 6" ou "Art 6"
    )

display_response(response, show_source=False)

Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: search
Action Input: {'input': 'Art 6º do estatuto do IFSP'}
Observation: O IFSP tem os seguintes objetivos: I. ministrar educação profissional técnica de nível médio, prioritariamente na forma de cursos integrados, para os concluintes do ensino fundamental e para o público da Educação de Jovens e Adultos; II. ministrar cursos de formação inicial e continuada de trabalhadores, tendo como objetivo a capacitação, o aperfeiçoamento, a especialização e a atualização de profissionais, em todos os níveis de escolaridade, nas áreas da educação profissional e tecnológica; III. realizar pesquisas, estimulando o desenvolvimento de soluções técnicas e tecnológicas, estendendo seus benefícios à comunidade; IV. desenvolver atividades de extensão de acordo com os princípios e finalidades da educação profissional e tecnológica, em articulação com o mundo do trabalho e os segmentos so

**`Final Response:`** O Art. 6º do estatuto do IFSP apresenta os objetivos institucionais, que incluem ministrar educação profissional, realizar pesquisas, desenvolver atividades de extensão, estimular processos educativos, desenvolver programas de extensão e divulgação cultural científica e tecnológica, realizar e estimular a pesquisa, produção cultural, empreendedorismo, cooperativismo e desenvolvimento científico e tecnológico, e promover a produção, desenvolvimento e transferência de tecnologias sociais.

In [ ]:
display_prompt_dict(agent.get_prompts())